# Plotting pre-processed SST1 RSoXS data
Use pyhyper's new RSoXS plotting shortctus

## Imports

In [ ]:
## Imports
import PyHyperScattering as phs
import pathlib
import sys
import io
import ast
import json
import datetime
import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from tqdm.auto import tqdm
import subprocess
import gc

print(f'Using PyHyperScattering Version: {phs.__version__}')

## Define paths

In [ ]:
notebookPath = pathlib.Path.cwd() 
zarrsPath = notebookPath  # use this if the example data is in the same folder as this notebook
# zarrsPath = pathlib.Path('/nsls2/data/sst/proposals/2024-2/pass-313412/rsoxs_hackathon_2024_data')  # use this if working in the NSLS2 Jupyterhub to access zarrs

## Load cartesian data 

In [ ]:
[f.name for f in zarrsPath.glob('*zarr')]

In [ ]:
DS = xr.open_zarr(zarrsPath.joinpath('cartesian_data_PM6_CF.zarr'))

# Compute any dask coordiantes
for coord_name, coord_data in DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        DS.coords[coord_name] = coord_data.compute()
        
DS

### Detector images

#### Facet plot at several selected energies

#### Single image at selected energy

#### Generate mp4 movie of detector image at all energies

## Load polar data & set plotting config

In [ ]:
[f.name for f in zarrsPath.glob('*zarr')]

In [ ]:
DS = xr.open_zarr(zarrsPath.joinpath('polar_data_PM6_CF.zarr'))

# Compute any dask coordiantes
for coord_name, coord_data in DS.coords.items():
    if isinstance(coord_data.data, da.Array):
        DS.coords[coord_name] = coord_data.compute()

# Select a dataarray (potentially from larger dataset)
pol = 0
data_variable = 'raw_intensity'
DA = DS[data_variable].sel(polarization=pol)

# Set default plot roi & hint settings:
DA.attrs['plot_roi'] = {
    'chi_width': 90,
    'q_range': (0.01, 0.09),
    'energy_range': (280, 295),
    'energy_default': 285,
    'selected_energies': np.array([275, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 300, 335])
}

DA.attrs['plot_hints'] = {
    'I_cmap': 'turbo',
    'xscale': 'log',
    'yscale': 'log'
}

DA

### Integrated scattered intensity (ISI) plots

In [ ]:
# Call ISI function, return ISI figure & axes object
# fig, ax = DA.pt.plot_ISI(save=False)
fig, ax = DA.pt.plot_ISI()

# Plot
plt.show()
plt.close('all')

### Intensity heatmaps

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_Imap(save=False)
fig, ax = DA.pt.plot_Imap()

# Plot
plt.show()
plt.close('all')

### Intensity vs Q linecuts

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_IvQ(save=False)
fig, ax = DA.pt.plot_IvQ()

# Plot
plt.show()
plt.close('all')

### Anisotropy heatmaps

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_ARmap(ar_vlim=0.4, save=False)
fig, ax = DA.pt.plot_ARmap(ar_vlim=0.4)

# Plot
plt.show()
plt.close('all')

### Anisotropy vs Q linecuts

In [ ]:
# Call Imap function, return Imap figure & axes list 
# fig, ax = DA.pt.plot_ARvQ(save=False)
fig, ax = DA.pt.plot_ARvQ()

# Plot
plt.show()
plt.close('all')

# Previous (and messy) detector plotting code for reference 

In [ ]:
# Parameters
edge = 'carbon'
intensity_type = 'raw'
pol = 0
energy = 285.2

for sample_name in selected_samples:
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    DA = DA.where(DA>0, 10)
    # Make & customize plot
    sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})

    cmin = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.03))
    cmax = float(sliced_DA.sel(qx=slice(0.01,0.05),qy=slice(0.01,0.05)).compute().quantile(0.999))
    
    # cmin = cmin if cmin > 1e8 else 1e8

    ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
    ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
    ax.figure.set_tight_layout(True)   
    ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
    ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=12)

    plt.show()
    plt.close('all')

In [ ]:
for pol in [0, 90]:
    # for sample_name in tqdm(DS.sample_name.values, desc=f'Polarization {pol}°'):
    for sample_name in tqdm(selected_samples, desc=f'Polarization {pol}°'):
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']

        savePath = plotsPath.joinpath('rsoxs_carbon_detector_movies_v1')
        savePath.mkdir(exist_ok=True)
        output_path = savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '16',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '22',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for i, energy in enumerate(tqdm(DA.energy.values, desc=f'Making the {sample_name} movie')):
            # Make & customize plot
            sliced_DA = DA.sel(polarization=pol).sel(energy=energy, method='nearest').swap_dims({'pix_x':'qx','pix_y':'qy'})
            
            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), cmap=cmap, norm=LogNorm(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Raw Intensity [arb. units]', rotation=270, labelpad=12)

            # Save first frame (or specific energy value)
            if energy == 285.2:
                ax.figure.savefig(savePath.joinpath(f'{sample_name}_{intensity_type}_pol{pol}deg.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")
            
        gc.collect()  # try to clear up some memory...
